<a href="https://colab.research.google.com/github/abelalanoca/P.MAGNO/blob/main/TP1/estructura_funcional_error_solo_en_cmac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------
# 📌 UNIFICADO: BALANCE GENERAL + ESTADO DE RESULTADOS (Múltiples archivos)
# --------------------------------------------------------
from google.colab import files, drive
import pandas as pd
import itertools
import re
from calendar import monthrange

# --------------------------------------------------------
# 🔹 Subir uno o varios archivos desde tu PC
# --------------------------------------------------------
uploaded = files.upload()

# 🔹 Montar Google Drive (opcional)
drive.mount("/content/drive")

# --------------------------------------------------------
# 🔹 Procesar cada archivo subido UNO POR UNO
# --------------------------------------------------------
for nombre_archivo in uploaded.keys():
    print(f"📂 Procesando archivo: {nombre_archivo}")

    # --------------------------------------------------------
    # 🔹 Leer hoja 1 del archivo subido (Balance)
    # --------------------------------------------------------
    df_raw = pd.read_excel(nombre_archivo, sheet_name=0)

    # --------------------------------------------------------
    # 🔹 Función: Normalizar nombres de bancos
    # --------------------------------------------------------
    def reemplazar_bancos_especificos(df):
        df_1 = df.copy()
        reemplazos = {
            "COMPARTAMOS BANCO": "Compartamos Banco*",
            "BANCO DE COMERCIO": "BANCOM"
        }
        df_1 = df_1.applymap(
            lambda x: reemplazos.get(str(x).strip().upper(), x) if isinstance(x, str) else x
        )
        return df_1

    df_1 = reemplazar_bancos_especificos(df_raw)

    # --------------------------------------------------------
    # 🔹 Función: Extraer listas de cuentas y bancos
    # --------------------------------------------------------
    def extraer_listas_balance(df):
        primera_col = df.iloc[:, 0].astype(str).str.strip().str.upper()

        # ---- Cuentas del Activo
        idx_activo = primera_col[primera_col == "ACTIVO"].index[0]
        idx_total_activo = primera_col[primera_col == "TOTAL ACTIVO"].index[0]
        cuentas_del_activo = (
            df.iloc[idx_activo:idx_total_activo + 1, 0]
            .dropna().astype(str).str.strip().tolist()
        )
        cuentas_del_activo = [c for c in cuentas_del_activo if c.upper() != "ACTIVO"]

        # ---- Cuentas del Pasivo
        idx_pasivo = primera_col[primera_col == "PASIVO"].index[0]
        idx_otras_conting = primera_col[primera_col == "OTRAS CUENTAS CONTINGENTES"].index[0]
        lista_pasivos = (
            df.iloc[idx_pasivo:idx_otras_conting + 1, 0]
            .dropna().astype(str).str.strip().tolist()
        )
        lista_pasivos = [p for p in lista_pasivos if p.upper() != "PASIVO"]

        # ---- Bancos (usando DISPONIBLE, 3 arriba y 1 a la derecha)
        idx_disponible = primera_col[primera_col == "DISPONIBLE"].index[0]
        fila_bancos = idx_disponible - 3
        bancos_activo = (
            df.iloc[fila_bancos, 1:]
            .dropna().astype(str).str.strip().tolist()
        )
        bancos_activo = [b for b in bancos_activo if b.upper() != "ACTIVO"]

        return cuentas_del_activo, lista_pasivos, bancos_activo

    cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_1)

    # --------------------------------------------------------
    # 🔹 Función: Extraer fecha y año desde nombre del archivo
    # --------------------------------------------------------
    def extraer_fecha_y_anio_desde_nombre(nombre_archivo):
        meses = {
            "en": (1, "Ene"), "fe": (2, "Feb"), "ma": (3, "Mar"), "ab": (4, "Apr"),
            "my": (5, "May"), "jn": (6, "Jun"), "jl": (7, "Jul"), "ag": (8, "Aug"),
            "se": (9, "Sep"), "oc": (10, "Oct"), "no": (11, "Nov"), "di": (12, "Dic")
        }
        nombre_lower = nombre_archivo.lower()
        mes_num, mes_abrev = None, None
        for abrev, (num, mes_str) in meses.items():
            if abrev in nombre_lower:
                mes_num, mes_abrev = num, mes_str
                break
        if mes_num is None:
            raise ValueError("No se encontró un mes válido en el nombre del archivo.")
        anio_match = re.search(r"(20\d{2})", nombre_lower)
        if anio_match:
            anio = int(anio_match.group(1))
        else:
            raise ValueError("No se encontró un año válido en el nombre del archivo.")
        ultimo_dia = monthrange(anio, mes_num)[1]
        return anio, f"{mes_abrev}-{ultimo_dia}"

    Anio, Fecha = extraer_fecha_y_anio_desde_nombre(nombre_archivo)

    # --------------------------------------------------------
    # 🔹 Función: Determinar ENTIDAD según nombre archivo
    # --------------------------------------------------------
    def obtener_entidad(nombre_archivo):
        entidad_dict = {
            "2201": "BANCO",
            "3101": "FINANCIERA",
            "1101": "CAJA MUNICIPAL",
            "2101": "CAJA RURAL",
            "4103": "EMPRESA DE CRÉDITO"
        }
        for clave, valor in entidad_dict.items():
            if clave in nombre_archivo:
                return valor
        return "DESCONOCIDO"

    Entidad = obtener_entidad(nombre_archivo)

    # --------------------------------------------------------
    # 🔹 Crear df_2 y llenarlo (idéntico a tu lógica)
    # --------------------------------------------------------
    def crear_df_2(Anio, Fecha, bancos_activo, cuentas_del_activo, lista_pasivos, Entidad):
        def limpiar_lista(lista):
            return [x for x in lista if pd.notnull(x) and str(x).strip() != ""]
        cuentas_del_activo = limpiar_lista(cuentas_del_activo)
        lista_pasivos = limpiar_lista(lista_pasivos)
        bancos_activo = limpiar_lista(bancos_activo)
        columnas_cuentas = cuentas_del_activo + lista_pasivos
        df_2 = pd.DataFrame(columns=["AÑO", "MES", "INSTITUCIÓN FINANCIERA", "MONEDA"] + columnas_cuentas + ["ENTIDAD"])
        entidades, monedas, meses, anios, tipo_entidad = [], [], [], [], []
        for banco in bancos_activo:
            for moneda in ["MN", "ME", "TOTAL"]:
                entidades.append(banco)
                monedas.append(moneda)
                meses.append(Fecha)
                anios.append(Anio)
                tipo_entidad.append(Entidad)
        df_2["AÑO"] = anios
        df_2["MES"] = meses
        df_2["INSTITUCIÓN FINANCIERA"] = entidades
        df_2["MONEDA"] = monedas
        df_2["ENTIDAD"] = tipo_entidad
        return df_2

    df_2 = crear_df_2(Anio, Fecha, bancos_activo, cuentas_del_activo, lista_pasivos, Entidad)

    def llenar_df_2_matriz(df_1, df_2, bancos, cuentas, tipo="ACTIVO"):
        df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
        idx_tipo = df_limpio[df_limpio.iloc[:, 0].str.upper() == tipo].index[0]
        filas_cuentas = {}
        for cuenta in cuentas:
            match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
            if match:
                filas_cuentas[cuenta] = match[0]
        columnas_bancos = {}
        for banco in bancos:
            match = df_limpio.columns[df_limpio.iloc[idx_tipo, :].astype(str) == banco].tolist()
            if match:
                columnas_bancos[banco] = df_limpio.columns.get_loc(match[0])
        for banco, col_idx in columnas_bancos.items():
            for cuenta, fila_idx in filas_cuentas.items():
                valores = df_1.iloc[fila_idx, col_idx:col_idx+3].tolist()
                for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                    df_2.loc[(df_2["INSTITUCIÓN FINANCIERA"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor
        return df_2

    df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, cuentas_del_activo, tipo="ACTIVO")
    df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, lista_pasivos, tipo="PASIVO")

    # --------------------------------------------------------
    # 🔹 Hoja 2 (Estado de Resultados) [idéntico]
    # --------------------------------------------------------
    df_raw_hoja2 = pd.read_excel(nombre_archivo, sheet_name=1)
    primera_col_hoja2 = df_raw_hoja2.iloc[:, 0].astype(str).str.strip()
    idx_ingresos = primera_col_hoja2[primera_col_hoja2.str.contains("INGRESOS FINANCIEROS", case=False, na=False)].index[0]
    idx_resultado = primera_col_hoja2[primera_col_hoja2.str.contains("RESULTADO NETO DEL EJERCICIO", case=False, na=False)].index[0]
    Cuentas_del_ER = df_raw_hoja2.iloc[idx_ingresos:idx_resultado + 1, 0].dropna().astype(str).str.strip().tolist()
    fila_bancos = idx_ingresos - 3
    Bancos_del_ER = (
        df_raw_hoja2.iloc[fila_bancos, 1:]
        .dropna().astype(str).str.strip().tolist()
    )
    columnas_ER = ["AÑO", "MES", "INSTITUCIÓN FINANCIERA", "MONEDA"] + Cuentas_del_ER + ["ENTIDAD"]
    df_3 = pd.DataFrame(columns=columnas_ER)
    entidades = list(itertools.chain.from_iterable([[banco]*3 for banco in Bancos_del_ER]))
    monedas = ["MN", "ME", "TOTAL"] * len(Bancos_del_ER)
    meses = [Fecha] * len(entidades)
    anios = [Anio] * len(entidades)
    tipo_entidad = [Entidad] * len(entidades)
    df_3["AÑO"] = anios
    df_3["MES"] = meses
    df_3["INSTITUCIÓN FINANCIERA"] = entidades
    df_3["MONEDA"] = monedas
    df_3["ENTIDAD"] = tipo_entidad

    def normalizar(texto):
        texto = str(texto).upper()
        texto = re.sub(r"[ÁÀÄÂ]", "A", texto)
        texto = re.sub(r"[ÉÈËÊ]", "E", texto)
        texto = re.sub(r"[ÍÌÏÎ]", "I", texto)
        texto = re.sub(r"[ÓÒÖÔ]", "O", texto)
        texto = re.sub(r"[ÚÙÜÛ]", "U", texto)
        texto = re.sub(r"[\(\)]", "", texto)
        texto = re.sub(r"\s+", " ", texto)
        return texto.strip()

    df_limpio = df_raw_hoja2.copy().astype(str).apply(lambda col: col.str.strip())
    df_limpio.iloc[:, 0] = df_limpio.iloc[:, 0].map(normalizar)

    for banco in Bancos_del_ER:
        banco_norm = normalizar(banco)
        mask_fila_banco = df_limpio.apply(
            lambda row: row.astype(str).map(normalizar).str.contains(banco_norm, regex=False).any(),
            axis=1
        )
        if not mask_fila_banco.any():
            continue
        fila_banco_idx = mask_fila_banco[mask_fila_banco].index[0]
        for cuenta in Cuentas_del_ER:
            cuenta_norm = normalizar(cuenta)
            mask_cuenta = df_limpio.iloc[:, 0].str.contains(cuenta_norm, regex=False)
            if not mask_cuenta.any():
                continue
            fila_cuenta_idx = mask_cuenta[mask_cuenta].index[0]
            mask_col = df_limpio.iloc[fila_banco_idx, :].map(normalizar).str.contains(banco_norm, regex=False)
            if not mask_col.any():
                continue
            col_banco_idx = df_limpio.columns.get_loc(mask_col[mask_col].index[0])
            valores = df_raw_hoja2.iloc[fila_cuenta_idx, col_banco_idx:col_banco_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_3.loc[(df_3["INSTITUCIÓN FINANCIERA"] == banco) & (df_3["MONEDA"] == moneda), cuenta] = valor

    # --------------------------------------------------------
    # 🔹 Exportar a Google Sheets (append sin borrar lo existente)
    # --------------------------------------------------------
    import gspread
    from google.auth import default
    from google.colab import auth
    from gspread_dataframe import set_with_dataframe

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    url = "https://docs.google.com/spreadsheets/d/1Kr9rfHkyLZX8DbhJ-166BpetNGGYKtwy7u0s1hzzljw/edit?usp=sharing"
    spreadsheet = gc.open_by_url(url)

    def append_dataframe(worksheet, df_new):
        existing_values = worksheet.get_all_values()
        if len(existing_values) == 0 or all(v == "" for v in existing_values[0]):
            set_with_dataframe(worksheet, df_new, include_index=False, include_column_header=True)
        else:
            next_row = len(existing_values) + 1
            set_with_dataframe(
                worksheet,
                df_new,
                row=next_row,
                include_index=False,
                include_column_header=False
            )

    try:
        hoja_balance = spreadsheet.worksheet("Balance_General")
    except:
        hoja_balance = spreadsheet.add_worksheet(title="Balance_General", rows="100", cols="50")
    append_dataframe(hoja_balance, df_2)

    try:
        hoja_er = spreadsheet.worksheet("Estado_de_Resultados")
    except:
        hoja_er = spreadsheet.add_worksheet(title="Estado_de_Resultados", rows="100", cols="50")
    append_dataframe(hoja_er, df_3)

    print(f"✅ Archivo {nombre_archivo} procesado y exportado correctamente.\n")


Saving B-2201-oc2015.XLS to B-2201-oc2015 (1).XLS
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Procesando archivo: B-2201-oc2015 (1).XLS


/tmp/ipython-input-4234251614.py:38: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_1 = df_1.applymap(
/tmp/ipython-input-4234251614.py:156: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-4234251614.py:156: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)


IndexError: index 0 is out of bounds for axis 0 with size 0